In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams["text.usetex"] = True

In [ ]:
gal_id, gmag, gerr, rmag, rerr, imag, ierr, zmag, zerr, _, zs = np.loadtxt("UGC9412_GRIZ_LPin.in", unpack=True)

In [ ]:
grcol = gmag - rmag
grerr = np.sqrt(gerr**2 + rerr**2)

In [ ]:
ricol = rmag - imag
rierr = np.sqrt(rerr**2 + ierr**2)

In [ ]:
izcol = imag - zmag
izerr = np.sqrt(zerr**2 + zerr**2)

In [ ]:
ref_z = 0.0311981
ref_mags = np.array([14.130, 13.688, 13.463, 13.297])
ref_cols = np.array([m1 - m2 for (m1, m2) in zip(ref_mags[:-1], ref_mags[1:])])

In [ ]:
import os

In [ ]:
filt_dir = os.path.abspath("Filters")
sed_dir = os.path.abspath("SEDs")

In [ ]:
from sedpy import observate

In [ ]:
observate.list_available_filters()

In [ ]:
filt_list_sdssp = observate.load_filters(["sdss_gp", "sdss_rp", "sdss_ip", "sdss_zp"], directory=filt_dir)

In [ ]:
filt_list_sdss0 = observate.load_filters(["sdss_g0", "sdss_r0", "sdss_i0", "sdss_z0"])

In [ ]:
for filt, f0 in zip(filt_list_sdssp, filt_list_sdss0):
    plt.fill_between(filt.wavelength, filt.transmission, label=filt.name, alpha=0.5)
    plt.plot(f0.wavelength, f0.transmission, label=f0.name)
plt.legend()
plt.xlabel("Longueur d'onde $\mathrm{[\AA]}$")
plt.ylabel("Transmission du filtre")

In [ ]:
SED_list = os.path.join(sed_dir, "CWW_KINNEY", "CWW_MOD.list")
os.path.isfile(SED_list)

In [ ]:
rpfilt = filt_list_sdssp[1]
f, a = plt.subplots(1, 1)
for filt in filt_list_sdssp:
    a.fill_between(filt.wavelength, filt.transmission, label=filt.name, alpha=0.5)
a.set_xlabel("Longueur d'onde $\mathrm{[\AA]}$")
a.set_ylabel("Transmission SDSS")
abis = a.twinx()
with open(SED_list, "r") as sedin:
    for ix, line in enumerate(sedin.readlines()):
        sed = line.split(" ")[0]
        fsed = os.path.join(sed_dir, sed)
        sed_fn = os.path.basename(fsed)
        wl, flux = np.loadtxt(fsed, unpack=True)
        sed_rmag = rpfilt.ab_mag(wl, flux)
        sed_fact = np.power(10, -0.4 * np.median(rmag)) / np.power(10, -0.4 * sed_rmag)
        name = "-".join(sed_fn.split("_")[0:2])
        sed_rmag_scale = rpfilt.ab_mag(wl, flux * sed_fact)
        # print(sed_rmag_scale)
        abis.semilogy(wl, flux * sed_fact + ix * 0.25e-14, label=name)
abis.set_ylabel("Densité spectrale de flux $\mathrm{[erg.cm^{-2}.s^{-1}.\AA^{-1}]}$")
abis.set_xlim(3500, 11100)
abis.set_ylim(3e-15, 2e-13)
f.legend(loc="lower left", bbox_to_anchor=(1.05, 0.0))

In [ ]:
def col_sed_at_z(z, sed_wl_rest, sed_fl, filters_list):
    wl_obs = (1 + z) * sed_wl_rest
    obs_mags = observate.getSED(wl_obs, sed_fl, filterlist=filters_list)
    obs_cols = np.array([m1 - m2 for (m1, m2) in zip(obs_mags[:-1], obs_mags[1:])])
    return obs_cols

In [ ]:
cols_ugc9412 = np.column_stack([grcol, ricol, izcol])
colserr_ugc9412 = np.column_stack([grerr, rierr, izerr])
cols_ugc9412.shape

In [ ]:
rpfilt = filt_list_sdssp[1]
f, a = plt.subplots(1, 1)
wlmeans = np.array([filt.wave_effective for filt in filt_list_sdssp])
xcols = np.array([0.5 * (w1 + w2) for (w1, w2) in zip(wlmeans[:-1], wlmeans[1:])])
for i, idgal in enumerate(gal_id):
    a.errorbar(xcols, cols_ugc9412[i, :], yerr=colserr_ugc9412[i, :], marker="x", label=f"UGC 9412 - {idgal:.0f}")
with open(SED_list, "r") as sedin:
    for line in sedin.readlines():
        sed = line.split(" ")[0]
        fsed = os.path.join(sed_dir, sed)
        sed_fn = os.path.basename(fsed)
        wl, flux = np.loadtxt(fsed, unpack=True)
        sed_rmag = rpfilt.ab_mag(wl, flux)
        sed_fact = np.power(10, -0.4 * np.median(rmag)) / np.power(10, -0.4 * sed_rmag)
        name = "-".join(sed_fn.split("_")[0:2])
        sed_rmag_scale = rpfilt.ab_mag(wl, flux * sed_fact)
        sed_cols = col_sed_at_z(zs[0], wl, flux * sed_fact, filt_list_sdssp)
        a.scatter(xcols, sed_cols, label=name, s=25)
a.scatter(xcols, ref_cols, marker="s", edgecolor="k", facecolor="none", label="Référence", s=81)
a.set_ylabel("Indices de couleur $[\mathrm{AB\ mags}]$")
a.set_xlabel("Longueur d'onde $0.5 \\times \left( \lambda_{mean}^{filtre\ j} + \lambda_{mean}^{filtre\ j+1} \\right)\ \mathrm{[\AA]}$")
a.legend(loc="lower left", bbox_to_anchor=(1.01, 0.0))
a.grid()

In [ ]:
from collections import namedtuple
import astropy.units as u

Template = namedtuple("Template", ["name", "wavelength", "flux_cgs"])
U_FL = u.def_unit("erg . cm^{-2} . s^{-1} . AA^{-1}", u.erg / (u.cm**2 * u.s * u.AA))

In [ ]:
templates_list = []
with open(SED_list, "r") as sedin:
    for line in sedin.readlines():
        sed = line.split(" ")[0]
        fsed = os.path.join(sed_dir, sed)
        sed_fn = os.path.basename(fsed)
        wl, flux = np.loadtxt(fsed, unpack=True)
        sed_rmag = rpfilt.ab_mag(wl, flux)
        sed_fact = np.power(10, -0.4 * np.median(rmag)) / np.power(10, -0.4 * sed_rmag)
        sedname = "-".join(sed_fn.split("_")[0:2])
        sed_rmag_scale = rpfilt.ab_mag(wl, flux * sed_fact)
        templates_list.append(Template(name=sedname, wavelength=wl * u.AA, flux_cgs=flux * sed_fact * U_FL))

In [ ]:
def chi2(cols, cols_err, template, redz, filter_list):
    sed_cols = col_sed_at_z(redz, template.wavelength.value, template.flux_cgs.value, filter_list)
    terms = np.power((sed_cols - cols) / cols_err, 2)
    # print(terms.shape)
    return np.sum(np.power((sed_cols - cols) / cols_err, 2)) / len(cols)

In [ ]:
z_grid = np.arange(0.0, 0.05, 0.0001)

In [ ]:
all_chisq = []
for templ in templates_list:
    chisq = []
    for ztest in z_grid:
        chival = chi2(cols_ugc9412, colserr_ugc9412, templ, ztest, filt_list_sdssp)
        chisq.append(chival)
    all_chisq.append(np.array(chisq) / 100000)
print(np.min(all_chisq))
exp = np.exp(-0.5 * np.array(all_chisq))
marg_dist = np.sum(exp, axis=0) / exp.shape[0]
norm = np.trapezoid(marg_dist, x=z_grid)
pdz = exp / norm
for i, templ in enumerate(templates_list):
    plt.plot(z_grid, pdz[i, :], label=templ.name)
plt.axvline(zs[0], color="k", label=f"$z_s={ref_z:.4f}$")
plt.xlabel("Redshift $z$")
plt.ylabel("$P(z|\mathrm{modele})$")
plt.legend()

In [ ]:
plt.plot(z_grid, marg_dist / norm)
plt.axvline(zs[0], color="k", label=f"$z_s={ref_z:.4f}$")
plt.xlabel("Redshift $z$")
plt.ylabel("Fonction densité de probabilité")
plt.legend()

In [ ]:
z_grid = np.arange(0.0, 1.0, 0.0001)
all_chisq = []
for templ in templates_list:
    chisq = []
    for ztest in z_grid:
        chisq.append(chi2(cols_ugc9412, colserr_ugc9412, templ, ztest, filt_list_sdssp))
    all_chisq.append(np.array(chisq) / 100000)
print(np.min(all_chisq))
exp = np.exp(-0.5 * np.array(all_chisq))
marg_dist = np.sum(exp, axis=0) / exp.shape[0]
norm = np.trapezoid(marg_dist, x=z_grid)
pdz = exp / norm
for i, templ in enumerate(templates_list):
    plt.plot(z_grid, pdz[i, :], label=templ.name)
plt.axvline(zs[0], color="k", label=f"$z_s={ref_z:.4f}$")
plt.xlabel("Redshift $z$")
plt.ylabel("$P(z|\mathrm{SED})$")
plt.legend(loc="lower right")
plt.grid()

In [ ]:
plt.plot(z_grid, marg_dist / norm)
plt.axvline(zs[0], color="k", label=f"$z_s={ref_z:.4f}$")
plt.xlabel("Redshift $z$")
plt.ylabel("Fonction densité de probabilité")
plt.legend()
plt.grid()

In [ ]:
z_grid = np.arange(0.0, 1.0, 0.001)
all_exps = []
for templ in templates_list:
    all_chisq = []
    for it, mn in enumerate(gal_id):
        chisq = []
        for ztest in z_grid:
            chisq.append(chi2(cols_ugc9412[it], colserr_ugc9412[it], templ, ztest, filt_list_sdssp))
        all_chisq.append(np.array(chisq) / 100000)
    exp = np.exp(-0.5 * np.array(all_chisq))
    all_exps.append(exp)

all_exps = np.array(all_exps)
# print(all_exps.shape)
marg_templ = np.sum(all_exps, axis=0) / all_exps.shape[0]
# print(marg_templ.shape)
marg_dist = np.sum(marg_templ, axis=0) / marg_templ.shape[0]
# print(marg_dist.shape)
norm = np.trapezoid(marg_dist, x=z_grid)
pdz = all_exps / norm
# print(pdz.shape)
for i, templ in enumerate(templates_list):
    pdz_mean = np.mean(pdz[i, :, :], axis=0)
    # print(pdz_mean.shape)
    pdz_std = np.std(all_exps[i, :, :], axis=0)
    plt.plot(z_grid, pdz_mean, label=templ.name)
    plt.fill_between(z_grid, pdz_mean - pdz_std, pdz_mean + pdz_std, alpha=0.2)
plt.axvline(zs[0], color="k", label=f"$z_s={ref_z:.4f}$")
plt.xlabel("Redshift $z$")
plt.ylabel("$p\left( z|\mathrm{SED} \\right)$")
plt.legend(loc="lower right")
plt.grid()
plt.show()


marg_mean = np.mean(marg_templ, axis=0) / norm
marg_std = np.std(marg_templ, axis=0) / norm
plt.plot(z_grid, marg_mean)
plt.fill_between(z_grid, marg_mean - marg_std, marg_mean + marg_std, alpha=0.2)
plt.axvline(zs[0], color="k", label=f"$z_s={ref_z:.4f}$")
plt.xlabel("Redshift $z$")
plt.ylabel("Fonction densité de probabilité $p(z)$")
plt.legend(loc="lower right")
plt.grid()
plt.show()

In [ ]:
z_grid = np.arange(0.0, 0.2, 0.0001)
all_exps = []
for templ in templates_list[1:3] + templates_list[6:]:
    all_chisq = []
    for it, mn in enumerate(gal_id):
        chisq = []
        for ztest in z_grid:
            chisq.append(chi2(cols_ugc9412[it], colserr_ugc9412[it], templ, ztest, filt_list_sdssp))
        all_chisq.append(np.array(chisq) / 100000)
    exp = np.exp(-0.5 * np.array(all_chisq))
    all_exps.append(exp)

all_exps = np.array(all_exps)
# print(all_exps.shape)
marg_templ = np.sum(all_exps, axis=0) / all_exps.shape[0]
# print(marg_templ.shape)
marg_dist = np.sum(marg_templ, axis=0) / marg_templ.shape[0]
# print(marg_dist.shape)
norm = np.trapezoid(marg_dist, x=z_grid)
pdz = all_exps / norm
# print(pdz.shape)
for i, templ in enumerate(templates_list[1:3] + templates_list[6:]):
    pdz_mean = np.mean(pdz[i, :, :], axis=0)
    # print(pdz_mean.shape)
    pdz_std = np.std(all_exps[i, :, :], axis=0)
    plt.plot(z_grid, pdz_mean, label=templ.name)
    plt.fill_between(z_grid, pdz_mean - pdz_std, pdz_mean + pdz_std, alpha=0.2)
plt.axvline(zs[0], color="k", label=f"$z_s={ref_z:.4f}$")
plt.xlabel("Redshift $z$")
plt.ylabel("$p\left( z|\mathrm{SED} \\right)$")
plt.legend(loc="lower right")
plt.grid()
plt.show()

marg_mean = np.mean(marg_templ, axis=0) / norm
marg_std = np.std(marg_templ, axis=0) / norm
zp = z_grid[np.argmax(marg_mean)]
print(f"$z_p={zp:.4f}$")

plt.plot(z_grid, marg_mean, label="Distribution marginalisée", lw=4)
# plt.fill_between(z_grid, marg_mean-marg_std, marg_mean+marg_std, alpha=0.2)
all_zp = []
for it, mn in enumerate(gal_id):
    plt.plot(z_grid, marg_templ[it, :] / norm, label=f"UGC 9412 - {mn:.0f}")
    all_zp.append(z_grid[np.argmax(marg_templ[it, :] / norm)])
print(f"z_p moyen = {np.mean(all_zp):.4f}, +/- {np.std(all_zp):.4f}")
# plt.plot(z_grid, marg_dist/norm)
plt.axvline(zs[0], color="k", ls="-", label=f"$z_s={ref_z:.4f}$")
plt.axvline(zp, color="k", ls=":", lw=3, label=f"$z_p={zp:.4f}$")
plt.xlabel("Redshift $z$")
plt.ylabel("Fonction densité de probabilité $p(z)$")
plt.legend(loc="lower right")
plt.grid()
plt.show()

In [ ]:
biais = (zp - ref_z) / (1 + ref_z)
print(0.85 * ref_z, biais, 1.15 * ref_z)

In [ ]:
zp = np.trapezoid(marg_mean * z_grid, x=z_grid)
print(f"$z_p={zp:.4f}$")

In [ ]:
z_grid = np.arange(0.0, 0.05, 0.0001)
all_chisq = []
for templ in templates_list:
    chisq = []
    for ztest in z_grid:
        chisq.append(chi2(ref_cols, colserr_ugc9412, templ, ztest, filt_list_sdssp))
    all_chisq.append(np.array(chisq) / 100000)
print(np.min(all_chisq))
exp = np.exp(-0.5 * np.array(all_chisq))
marg_dist = np.sum(exp, axis=0) / exp.shape[0]
norm = np.trapezoid(marg_dist, x=z_grid)
pdz = exp / norm
for i, templ in enumerate(templates_list):
    plt.plot(z_grid, pdz[i, :], label=templ.name)
plt.axvline(ref_z, color="k", label=f"$z_s={ref_z:.4f}$")
plt.xlabel("Redshift $z$")
plt.ylabel("$P(z|\mathrm{modele})$")
plt.legend()

In [ ]:
plt.plot(z_grid, marg_dist / norm)
plt.axvline(ref_z, color="k", label=f"$z_s={ref_z:.4f}$")
plt.xlabel("Redshift $z$")
plt.ylabel("Fonction densité de probabilité")
plt.legend()

In [ ]:
from astroquery.simbad import Simbad

simbad = Simbad()
# simbad.add_votable_fields('measurements')
simbad.add_votable_fields("z_value")
result = simbad.query_object("UGC 9412")
result

In [ ]:
simbad.list_votable_fields()

In [ ]:
tab63 = np.array([[6, 5, 4], [6, 5, 4], [6, 5, 4], [6, 5, 4], [6, 5, 4], [6, 5, 4]])
tab63.shape

In [ ]:
tab3 = np.array([1, 2, 3])
tab3.shape

In [ ]:
tab63 - tab3

In [ ]:
tab36 = np.transpose(tab63)
tab36.shape

## Spectre

In [ ]:
spec_file = "UGC_9412_wlc.fits"  #'UGC_9412_wlc_cont.fits' #

In [ ]:
from astropy.io import fits

with fits.open(spec_file) as hdul:
    hdu = hdul[0]
    header = hdu.header
    data = hdu.data
header

In [ ]:
print(data.shape)

In [ ]:
wls = np.array([header["CRVAL1"] + i * header["CDELT1"] for i in range(len(data))])
print(len(wls))

In [ ]:
plt.plot(wls, data)

In [ ]:
f, a = plt.subplots(1, 1)
for filt in filt_list_sdssp:
    a.fill_between(filt.wavelength, filt.transmission, label=filt.name, alpha=0.5)
a.set_xlabel("Longueur d'onde $\mathrm{[\AA]}$")
a.set_ylabel("Transmission SDSS")
abis = a.twinx()
for i, templ in enumerate(templates_list[1:3] + templates_list[6:]):
    abis.plot(templ.wavelength, templ.flux_cgs, label=templ.name)
spec_rmag = rpfilt.ab_mag(wls, data)
spec_fact = np.power(10, -0.4 * np.median(rmag)) / np.power(10, -0.4 * spec_rmag)
ref_sed = data * spec_fact
abis.plot(wls / (1 + ref_z), ref_sed, label="UGC 9412")
abis.set_ylabel("Densité spectrale de flux $\mathrm{[erg.cm^{-2}.s^{-1}.\AA^{-1}]}$")
abis.set_xlim(3500, 11100)
# abis.set_ylim(3e-15, 2e-13)
f.legend(loc="lower left", bbox_to_anchor=(1.05, 0.0))

In [ ]:
gpfilt = filt_list_sdssp[0]
rpfilt = filt_list_sdssp[1]
ipfilt = filt_list_sdssp[2]

spec_gmag = gpfilt.ab_mag(wls, data)
spec_fact_g = np.power(10, -0.4 * np.median(gmag)) / np.power(10, -0.4 * spec_gmag)

spec_rmag = rpfilt.ab_mag(wls, data)
spec_fact_r = np.power(10, -0.4 * np.median(rmag)) / np.power(10, -0.4 * spec_rmag)

spec_imag = ipfilt.ab_mag(wls, data)
spec_fact_i = np.power(10, -0.4 * np.median(imag)) / np.power(10, -0.4 * spec_imag)


def calib_fun(lamb, a, b, c):
    return a * np.power(lamb, 2) + b * lamb + c


lambref = wlmeans[:3]
calib_vals = np.array([spec_fact_g, spec_fact_r, spec_fact_i])

from scipy.optimize import curve_fit

popt, pcov = curve_fit(calib_fun, lambref, calib_vals)

f, a = plt.subplots(1, 1)
a.plot(wls, data, c="tab:orange", label="Spectre UGC 9412\n Crédit : Emmanuel Valentin")
abis = a.twinx()
abis.plot(wls, calib_fun(wls, *popt), label="Polynôme du second degré de\n calibration en intensité")
abis.scatter(lambref, calib_vals, marker="x", label="Points d'étalonnage sur\n la photométrie gp, rp, ip")
a.set_xlabel("Longueur d'onde $\mathrm{[\AA]}$")
a.set_ylabel("Spectre brut $\mathrm{[ADU]}$")
abis.set_ylabel("Fonction de calibration $\mathrm{[erg.cm^{-2}.s^{-1}.\AA^{-1}.ADU^{-1}]}$")
f.legend(loc="upper left")

In [ ]:
f, a = plt.subplots(1, 1)
for filt in filt_list_sdssp:
    a.fill_between(filt.wavelength, filt.transmission, label=filt.name, alpha=0.5)
a.set_xlabel("Longueur d'onde $\mathrm{[\AA]}$")
a.set_ylabel("Transmission SDSS")
abis = a.twinx()
for i, templ in enumerate(templates_list[1:3] + templates_list[6:]):
    abis.plot(templ.wavelength, templ.flux_cgs, label=templ.name, lw=1, alpha=0.7)

ref_sed = data * calib_fun(wls, *popt)
abis.plot(wls / (1 + ref_z), ref_sed, c="k", lw=1.3, label="UGC 9412 à $z=0$", alpha=0.8)
abis.plot(wls, ref_sed, c="gray", ls=":", lw=1.3, label="UGC 9412 - original", alpha=0.7)
abis.set_ylabel("Densité spectrale de flux $\mathrm{[erg.cm^{-2}.s^{-1}.\AA^{-1}]}$")
abis.set_xlim(3500, 9100)
# abis.set_ylim(3e-15, 2e-13)
abis.grid()
f.legend(loc="lower left", bbox_to_anchor=(0.97, 0.05))